In [4]:
import tensorflow as tf
tf.__version__


'2.12.0'

In [5]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [6]:
subject = 'Ceará - Consumo de Cimento (t)'
split_index = 6 #Referente aos 7 anos de input  

In [7]:
data = pd.read_csv('../Inputs Atuais/2014_01_model_input_CE.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data

,Ano,Ceará - IDH Educacao,Ceará - IDH,Ceará - PIB - Estadual,Ceará - PIB - Construção Civil,Ceará - PIB - Per Capita,Ceará - PIB - Preços de Mercado,Ceará - Produção de Cimento (t),Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Ceará - Desemprego,Ceará - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Ceará - Consumo de Cimento (t)
0,2014,0.709103,0.714492,8.275393e+07,4.756888e+06,7.821411,6.627704e+07,1936.472431,5.676413e+08,2.779340e+09,...,10.652422,0.790518,18.599712,1009.682618,1313.782408,27.429527,21.316386,70.443006,2.652161,1964.413143
1,2015,0.727986,0.728548,8.340186e+07,4.705636e+06,7.807860,6.621518e+07,2117.547255,5.179392e+08,2.561731e+09,...,11.020421,0.803775,18.179780,1009.682825,1316.471785,27.559921,21.342508,70.089489,2.538890,2037.703714
2,2016,0.731530,0.729780,8.386853e+07,4.627836e+06,7.802392,6.619932e+07,2241.119301,4.524144e+08,2.516859e+09,...,11.073199,0.803391,19.751583,1009.575611,1278.441481,27.460532,21.334656,70.525466,2.458740,2146.547143
3,2017,0.733669,0.730487,8.401644e+07,4.573568e+06,7.799909,6.618240e+07,2243.972991,4.213689e+08,2.497710e+09,...,10.712887,0.802608,21.401573,1009.628585,1268.231186,27.368964,21.410344,71.183860,2.400073,2035.158000
4,2018,0.735366,0.730532,8.375716e+07,4.578630e+06,7.786499,6.603768e+07,2244.384216,4.166580e+08,2.797145e+09,...,10.581166,0.801391,22.383595,1009.641333,1238.329944,27.337958,21.546608,71.879998,2.343412,1758.900000
5,2019,0.734945,0.729762,8.307654e+07,4.576925e+06,7.763872,6.580977e+07,2418.807067,4.462625e+08,3.484252e+09,...,10.594941,0.799790,21.406645,1009.484821,1207.516910,27.349217,21.572987,71.967704,2.386761,1701.230000
6,2020,0.734485,0.728468,8.195272e+07,4.597074e+06,7.737276,6.551228e+07,2641.331290,5.489412e+08,4.114911e+09,...,10.540017,0.797680,21.139362,1009.280982,1199.667493,27.383165,21.576889,71.875921,2.420873,1864.364000
7,2021,0.731620,0.726056,8.044118e+07,4.615043e+06,7.722327,6.527597e+07,2791.618748,6.638811e+08,4.421848e+09,...,10.393445,0.795469,20.964804,1009.206646,1202.013627,27.355353,21.510868,71.733764,2.427626,2127.104000
8,2022,0.728495,0.725333,7.860880e+07,4.638968e+06,7.704053,6.498173e+07,2845.448417,7.010332e+08,4.533216e+09,...,10.367164,0.796512,20.315206,1009.229891,1205.231881,27.293728,21.413639,71.569256,2.436736,1966.646000


In [8]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Ceará - IDH Educacao,Ceará - IDH,Ceará - PIB - Estadual,Ceará - PIB - Construção Civil,Ceará - PIB - Per Capita,Ceará - PIB - Preços de Mercado,Ceará - Produção de Cimento (t),Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Ceará - IDH Renda,Ceará - Desemprego,Ceará - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,0.709103,0.714492,8.275393e+07,4.756888e+06,7.821411,6.627704e+07,1936.472431,5.676413e+08,2.779340e+09,1.954361e+10,...,0.650584,10.652422,0.790518,18.599712,1009.682618,1313.782408,27.429527,21.316386,70.443006,2.652161
1,0.727986,0.728548,8.340186e+07,4.705636e+06,7.807860,6.621518e+07,2117.547255,5.179392e+08,2.561731e+09,1.327596e+10,...,0.661305,11.020421,0.803775,18.179780,1009.682825,1316.471785,27.559921,21.342508,70.089489,2.538890
2,0.731530,0.729780,8.386853e+07,4.627836e+06,7.802392,6.619932e+07,2241.119301,4.524144e+08,2.516859e+09,1.011370e+10,...,0.661696,11.073199,0.803391,19.751583,1009.575611,1278.441481,27.460532,21.334656,70.525466,2.458740
3,0.733669,0.730487,8.401644e+07,4.573568e+06,7.799909,6.618240e+07,2243.972991,4.213689e+08,2.497710e+09,8.456309e+09,...,0.662214,10.712887,0.802608,21.401573,1009.628585,1268.231186,27.368964,21.410344,71.183860,2.400073
4,0.735366,0.730532,8.375716e+07,4.578630e+06,7.786499,6.603768e+07,2244.384216,4.166580e+08,2.797145e+09,7.937760e+09,...,0.661870,10.581166,0.801391,22.383595,1009.641333,1238.329944,27.337958,21.546608,71.879998,2.343412
5,0.734945,0.729762,8.307654e+07,4.576925e+06,7.763872,6.580977e+07,2418.807067,4.462625e+08,3.484252e+09,1.091282e+10,...,0.661495,10.594941,0.799790,21.406645,1009.484821,1207.516910,27.349217,21.572987,71.967704,2.386761
6,0.734485,0.728468,8.195272e+07,4.597074e+06,7.737276,6.551228e+07,2641.331290,5.489412e+08,4.114911e+09,1.625333e+10,...,0.660085,10.540017,0.797680,21.139362,1009.280982,1199.667493,27.383165,21.576889,71.875921,2.420873


In [9]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0    2037.703714
1    2146.547143
2    2035.158000
3    1758.900000
4    1701.230000
5    1864.364000
6    2127.104000
7    1966.646000
8            NaN
Name: Ceará - Consumo de Cimento (t), dtype: float64

In [10]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Ceará - IDH Educacao,Ceará - IDH,Ceará - PIB - Estadual,Ceará - PIB - Construção Civil,Ceará - PIB - Per Capita,Ceará - PIB - Preços de Mercado,Ceará - Produção de Cimento (t),Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Ceará - IDH Renda,Ceará - Desemprego,Ceará - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-2.152600,-2.221518,-1.613704,1.695576,1.351471,1.002064,-1.794892,1.779974,0.019134,2.003355,...,-2.230779,-0.602947,-2.139102,-1.086763,0.966369,1.115010,0.153665,-1.012434,-0.794662,1.805383
1,-0.085464,0.222854,-0.171835,0.973244,0.601507,0.606733,-0.563381,0.870373,-0.621387,0.401153,...,0.347310,1.244791,0.776139,-1.357213,0.969371,1.184232,1.845870,-0.758615,-1.285865,0.722359
2,0.302522,0.437036,0.866669,-0.123236,0.298919,0.505408,0.277046,-0.328802,-0.753466,-0.407217,...,0.441423,1.509790,0.691680,-0.344921,-0.585087,0.205375,0.556042,-0.834906,-0.680085,-0.043974
3,0.536691,0.560029,1.195814,-0.888082,0.161523,0.397313,0.296455,-0.896969,-0.809829,-0.830898,...,0.565953,-0.299348,0.519578,0.717726,0.182961,-0.057426,-0.632312,-0.099473,0.234738,-0.604914
4,0.722484,0.567735,0.618825,-0.816742,-0.580621,-0.527548,0.299251,-0.983185,0.071542,-0.963455,...,0.483190,-0.960725,0.251865,1.350180,0.367800,-0.827050,-1.034686,1.224557,1.202005,-1.146660
5,0.676366,0.433865,-0.895770,-0.840759,-1.832799,-1.983971,1.485521,-0.441390,2.094006,-0.202939,...,0.392903,-0.891561,-0.100159,0.720993,-1.901414,-1.620142,-0.888580,1.480871,1.323870,-0.732193


In [11]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0    2037.703714
1    2146.547143
2    2035.158000
3    1758.900000
4    1701.230000
5    1864.364000
Name: Ceará - Consumo de Cimento (t), dtype: float64

In [12]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Ceará - IDH Educacao,Ceará - IDH,Ceará - PIB - Estadual,Ceará - PIB - Construção Civil,Ceará - PIB - Per Capita,Ceará - PIB - Preços de Mercado,Ceará - Produção de Cimento (t),Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Ceará - IDH Renda,Ceará - Desemprego,Ceará - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
6,0.564039,0.192696,-1.932435,-0.504445,-1.912148,-2.024596,1.836832,1.16956,2.035192,0.985175,...,0.049923,-0.988778,-0.510818,0.497547,-2.151029,-1.389375,-0.408959,1.21948,1.009217,-0.371562


In [13]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

6    2127.104
Name: Ceará - Consumo de Cimento (t), dtype: float64

In [31]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=500, verbose=1)
    # Método estocástico e learning rate=0.01
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(74, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(2048, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(512, activation='tanh'),
        tf.keras.layers.Dense(64, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=5000, 
                      batch_size=len(train_input), 
                      validation_data=(train_input.iloc[-1:], train_target.iloc[-1:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [32]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(10)]
    print(random_seeds)

    best_val_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, 0, seed)

        current_val_loss = min(history.history['val_loss'])
        print(f"val_loss: {current_val_loss}")

        if current_val_loss < best_val_loss:
            best_val_loss = current_val_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [33]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[1259375243, 2005145932, 1503939284, 3424124929, 3100042687, 2635104207, 804626077, 108872007, 954244436, 560737332]


Step: 0 ___________________________________________
Epoch 767: early stopping
val_loss: 0.2687723636627197
winner_seed: 1259375243


Step: 1 ___________________________________________
Epoch 1148: early stopping
val_loss: 1628.2879638671875


Step: 2 ___________________________________________
Epoch 1065: early stopping
val_loss: 0.7166597843170166


Step: 3 ___________________________________________
Epoch 1273: early stopping
val_loss: 0.6603546142578125


Step: 4 ___________________________________________
Epoch 1245: early stopping
val_loss: 42.536128997802734


Step: 5 ___________________________________________
Epoch 1159: early stopping
val_loss: 4193.416015625


Step: 6 ___________________________________________
Epoch 1126: early stopping
val_loss: 1833.2655029296875


Step: 7 ___________________________________________
Epoch 1408: early stopping
val_loss: 1.0

In [34]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/5000
1/1 [==============================] - 0s 416ms/step - loss: 3727481.2500 - val_loss: 3308930.2500
Epoch 2/5000
1/1 [==============================] - 0s 24ms/step - loss: 3611242.0000 - val_loss: 5998954.5000
Epoch 3/5000
1/1 [==============================] - 0s 22ms/step - loss: 3535656.0000 - val_loss: 6476129.0000
Epoch 4/5000
1/1 [==============================] - 0s 25ms/step - loss: 4188608.7500 - val_loss: 4280673.0000
Epoch 5/5000
1/1 [==============================] - 0s 25ms/step - loss: 3435016.0000 - val_loss: 3865905.2500
Epoch 6/5000
1/1 [==============================] - 0s 23ms/step - loss: 3350192.7500 - val_loss: 3609305.2500
Epoch 7/5000
1/1 [==============================] - 0s 23ms/step - loss: 3283155.2500 - val_loss: 3542419.2500
Epoch 8/5000
1/1 [==============================] - 0s 25ms/step - loss: 3103653.0000 - val_loss: 2796696.0000
Epoch 9/5000
1/1 [==============================] - 0s 22ms/step - loss: 2878834.0000 - val_loss: 2396548.2500


1/1 [==============================] - 0s 22ms/step - loss: 888472.6875 - val_loss: 1250936.8750
Epoch 75/5000
1/1 [==============================] - 0s 22ms/step - loss: 1149237.1250 - val_loss: 717953.6875
Epoch 76/5000
1/1 [==============================] - 0s 22ms/step - loss: 1065050.6250 - val_loss: 937504.2500
Epoch 77/5000
1/1 [==============================] - 0s 33ms/step - loss: 915177.8125 - val_loss: 666501.0000
Epoch 78/5000
1/1 [==============================] - 0s 30ms/step - loss: 773407.5625 - val_loss: 533686.0625
Epoch 79/5000
1/1 [==============================] - 0s 24ms/step - loss: 955539.2500 - val_loss: 790962.6875
Epoch 80/5000
1/1 [==============================] - 0s 23ms/step - loss: 869749.3125 - val_loss: 905113.3750
Epoch 81/5000
1/1 [==============================] - 0s 21ms/step - loss: 854944.5625 - val_loss: 590682.1250
Epoch 82/5000
1/1 [==============================] - 0s 22ms/step - loss: 837178.9375 - val_loss: 587570.6875
Epoch 83/5000
1/1 [==

1/1 [==============================] - 0s 23ms/step - loss: 225172.0781 - val_loss: 256505.4062
Epoch 149/5000
1/1 [==============================] - 0s 22ms/step - loss: 271210.1562 - val_loss: 307756.7812
Epoch 150/5000
1/1 [==============================] - 0s 24ms/step - loss: 235690.6719 - val_loss: 221612.6875
Epoch 151/5000
1/1 [==============================] - 0s 19ms/step - loss: 209989.9844 - val_loss: 161186.1719
Epoch 152/5000
1/1 [==============================] - 0s 27ms/step - loss: 194087.2500 - val_loss: 146720.0469
Epoch 153/5000
1/1 [==============================] - 0s 26ms/step - loss: 211742.0469 - val_loss: 171514.7812
Epoch 154/5000
1/1 [==============================] - 0s 22ms/step - loss: 208954.3125 - val_loss: 169493.1250
Epoch 155/5000
1/1 [==============================] - 0s 21ms/step - loss: 193536.8281 - val_loss: 149299.2188
Epoch 156/5000
1/1 [==============================] - 0s 22ms/step - loss: 224734.2500 - val_loss: 209081.7969
Epoch 157/5000
1

1/1 [==============================] - 0s 21ms/step - loss: 73286.0391 - val_loss: 22891.3359
Epoch 223/5000
1/1 [==============================] - 0s 23ms/step - loss: 56660.2383 - val_loss: 29333.9629
Epoch 224/5000
1/1 [==============================] - 0s 23ms/step - loss: 71227.0859 - val_loss: 15855.0586
Epoch 225/5000
1/1 [==============================] - 0s 26ms/step - loss: 81988.2891 - val_loss: 36793.3008
Epoch 226/5000
1/1 [==============================] - 0s 24ms/step - loss: 65274.5742 - val_loss: 21690.7246
Epoch 227/5000
1/1 [==============================] - 0s 22ms/step - loss: 52649.2344 - val_loss: 27432.3691
Epoch 228/5000
1/1 [==============================] - 0s 22ms/step - loss: 60655.7773 - val_loss: 28870.3320
Epoch 229/5000
1/1 [==============================] - 0s 21ms/step - loss: 62324.8633 - val_loss: 15136.2080
Epoch 230/5000
1/1 [==============================] - 0s 23ms/step - loss: 56343.5039 - val_loss: 12557.4297
Epoch 231/5000
1/1 [==============

1/1 [==============================] - 0s 26ms/step - loss: 28646.4473 - val_loss: 2062.6255
Epoch 299/5000
1/1 [==============================] - 0s 22ms/step - loss: 26598.0312 - val_loss: 795.6573
Epoch 300/5000
1/1 [==============================] - 0s 21ms/step - loss: 27381.7441 - val_loss: 2359.6089
Epoch 301/5000
1/1 [==============================] - 0s 26ms/step - loss: 27864.5176 - val_loss: 3990.6550
Epoch 302/5000
1/1 [==============================] - 0s 22ms/step - loss: 25640.1934 - val_loss: 1399.8578
Epoch 303/5000
1/1 [==============================] - 0s 29ms/step - loss: 23254.5410 - val_loss: 462.3865
Epoch 304/5000
1/1 [==============================] - 0s 23ms/step - loss: 29016.5996 - val_loss: 2133.5107
Epoch 305/5000
1/1 [==============================] - 0s 23ms/step - loss: 26083.0547 - val_loss: 2586.2041
Epoch 306/5000
1/1 [==============================] - 0s 23ms/step - loss: 31186.3203 - val_loss: 5519.3184
Epoch 307/5000
1/1 [=========================

...

Epoch 450/5000
1/1 [==============================] - 0s 23ms/step - loss: 16335.2686 - val_loss: 9766.2500
Epoch 451/5000
1/1 [==============================] - 0s 21ms/step - loss: 12607.4502 - val_loss: 6797.8335
Epoch 452/5000
1/1 [==============================] - 0s 23ms/step - loss: 13657.8330 - val_loss: 5586.5952
Epoch 453/5000
1/1 [==============================] - 0s 22ms/step - loss: 23874.8047 - val_loss: 1890.9440
Epoch 454/5000
1/1 [==============================] - 0s 22ms/step - loss: 13186.9658 - val_loss: 3405.1470
Epoch 455/5000
1/1 [==============================] - 0s 23ms/step - loss: 11480.9600 - val_loss: 3565.6875
Epoch 456/5000
1/1 [==============================] - 0s 23ms/step - loss: 16549.4824 - val_loss: 7008.4409
Epoch 457/5000
1/1 [==============================] - 0s 23ms/step - loss: 41634.6992 - val_loss: 14246.9521
Epoch 458/5000
1/1 [==============================] - 0s 21ms/step - loss: 29780.8496 - val_loss: 10802.1416
Epoch 459/5000
1/1 [======

1/1 [==============================] - 0s 23ms/step - loss: 15600.6875 - val_loss: 6343.4263
Epoch 526/5000
1/1 [==============================] - 0s 24ms/step - loss: 35956.3086 - val_loss: 5562.5342
Epoch 527/5000
1/1 [==============================] - 0s 23ms/step - loss: 21636.3730 - val_loss: 6509.7046
Epoch 528/5000
1/1 [==============================] - 0s 23ms/step - loss: 13586.6377 - val_loss: 6348.4248
Epoch 529/5000
1/1 [==============================] - 0s 22ms/step - loss: 23551.9219 - val_loss: 14445.5928
Epoch 530/5000
1/1 [==============================] - 0s 22ms/step - loss: 12648.6572 - val_loss: 9113.1865
Epoch 531/5000
1/1 [==============================] - 0s 22ms/step - loss: 14232.5869 - val_loss: 10704.4434
Epoch 532/5000
1/1 [==============================] - 0s 23ms/step - loss: 14317.8154 - val_loss: 14732.5430
Epoch 533/5000
1/1 [==============================] - 0s 19ms/step - loss: 23601.8027 - val_loss: 8763.2637
Epoch 534/5000
1/1 [====================

1/1 [==============================] - 0s 22ms/step - loss: 29519.8613 - val_loss: 13642.4512
Epoch 601/5000
1/1 [==============================] - 0s 24ms/step - loss: 17687.1719 - val_loss: 14974.3262
Epoch 602/5000
1/1 [==============================] - 0s 24ms/step - loss: 20593.5215 - val_loss: 9993.1406
Epoch 603/5000
1/1 [==============================] - 0s 22ms/step - loss: 22216.6875 - val_loss: 5982.6787
Epoch 604/5000
1/1 [==============================] - 0s 24ms/step - loss: 12494.6064 - val_loss: 3443.6067
Epoch 605/5000
1/1 [==============================] - 0s 21ms/step - loss: 17316.6602 - val_loss: 3227.0361
Epoch 606/5000
1/1 [==============================] - 0s 23ms/step - loss: 21072.8672 - val_loss: 3461.4666
Epoch 607/5000
1/1 [==============================] - 0s 24ms/step - loss: 33816.9062 - val_loss: 850.9342
Epoch 608/5000
1/1 [==============================] - 0s 22ms/step - loss: 14720.0029 - val_loss: 1824.8916
Epoch 609/5000
1/1 [======================

1/1 [==============================] - 0s 18ms/step - loss: 11057.4834 - val_loss: 75.4981
Epoch 677/5000
1/1 [==============================] - 0s 18ms/step - loss: 26781.4824 - val_loss: 190.7649
Epoch 678/5000
1/1 [==============================] - 0s 19ms/step - loss: 27833.9531 - val_loss: 2823.1736
Epoch 679/5000
1/1 [==============================] - 0s 19ms/step - loss: 16851.2539 - val_loss: 288.9709
Epoch 680/5000
1/1 [==============================] - 0s 19ms/step - loss: 9139.5186 - val_loss: 1794.0995
Epoch 681/5000
1/1 [==============================] - 0s 18ms/step - loss: 31595.4688 - val_loss: 3443.4060
Epoch 682/5000
1/1 [==============================] - 0s 17ms/step - loss: 9576.3877 - val_loss: 717.7585
Epoch 683/5000
1/1 [==============================] - 0s 21ms/step - loss: 16031.9639 - val_loss: 435.8217
Epoch 684/5000
1/1 [==============================] - 0s 20ms/step - loss: 24785.6270 - val_loss: 2574.6333
Epoch 685/5000
1/1 [==============================]

1/1 [==============================] - 0s 19ms/step - loss: 27564.3594 - val_loss: 846.3824
Epoch 753/5000
1/1 [==============================] - 0s 18ms/step - loss: 12430.6172 - val_loss: 2555.4309
Epoch 754/5000
1/1 [==============================] - 0s 18ms/step - loss: 15182.2939 - val_loss: 1509.1119
Epoch 755/5000
1/1 [==============================] - 0s 19ms/step - loss: 21925.6270 - val_loss: 2887.5012
Epoch 756/5000
1/1 [==============================] - 0s 18ms/step - loss: 13029.5088 - val_loss: 378.7518
Epoch 757/5000
1/1 [==============================] - 0s 20ms/step - loss: 12728.5234 - val_loss: 615.8480
Epoch 758/5000
1/1 [==============================] - 0s 19ms/step - loss: 20123.9492 - val_loss: 569.0717
Epoch 759/5000
1/1 [==============================] - 0s 19ms/step - loss: 15524.2031 - val_loss: 560.2883
Epoch 760/5000
1/1 [==============================] - 0s 19ms/step - loss: 4433.2466 - val_loss: 636.7059
Epoch 761/5000
1/1 [==============================

In [35]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 45ms/step
[2127.104](test_target) - [[1974.0724]](prediction) = 153.03161230468731


In [36]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [37]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Ceará - IDH Educacao,Ceará - IDH,Ceará - PIB - Estadual,Ceará - PIB - Construção Civil,Ceará - PIB - Per Capita,Ceará - PIB - Preços de Mercado,Ceará - Produção de Cimento (t),Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Ceará - IDH Renda,Ceará - Desemprego,Ceará - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-1.398853,-1.410491,-1.285482,1.132803,1.357976,1.387610,-1.294141,1.165029,1.396038,1.335266,...,-1.413532,-1.404811,-1.413755,-0.367178,0.705055,0.628087,-0.967279,-1.352188,0.477785,1.288373
1,0.519405,0.616443,0.132199,0.166789,-0.337043,-0.457367,0.153209,0.111769,-0.502296,-0.264151,...,0.668759,0.561410,0.738048,-0.999156,0.709156,0.783285,1.377104,1.034830,-1.391625,-0.139142
2,0.879448,0.794048,1.153282,-1.299592,-1.020932,-0.930244,1.140932,-1.276798,-0.893743,-1.071116,...,0.744773,0.843401,0.675707,1.366334,-1.414212,-1.411372,-0.409825,0.317358,0.913839,-1.149232


train_target:


,Ceará - Consumo de Cimento (t)
0,2037.703714
1,2146.547143
2,2035.158000


test_input:


...

,Ceará - IDH Educacao,Ceará - IDH,Ceará - PIB - Estadual,Ceará - PIB - Construção Civil,Ceará - PIB - Per Capita,Ceará - PIB - Preços de Mercado,Ceará - Produção de Cimento (t),Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Ceará - IDH Renda,Ceará - Desemprego,Ceará - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
6,0.564039,0.192696,-1.932435,-0.504445,-1.912148,-2.024596,1.836832,1.16956,2.035192,0.985175,...,0.049923,-0.988778,-0.510818,0.497547,-2.151029,-1.389375,-0.408959,1.21948,1.009217,-0.371562


test_target:


,Ceará - Consumo de Cimento (t)
6,2127.104


Epoch 767: early stopping
1/1 [==============================] - 0s 40ms/step
Error: 153.03161230468731




[2090.535400390625, 1880.240966796875, 1789.978515625, 1974.0723876953125]

In [38]:
display(targets)
display(predictions)

[1758.9, 1701.23, 1864.364, 2127.104]

[2090.535400390625, 1880.240966796875, 1789.978515625, 1974.0723876953125]

In [39]:
mae = mean_absolute_error(predictions, targets)
mae

184.5158659667968

In [40]:
porcentage = mae/np.mean(targets)
porcentage

0.09904767593034236